In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
train_datagen = ImageDataGenerator(
        rescale=1.0/255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(
        'Dataset/train',
        target_size=(150, 150),
        batch_size=10)
test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
        'Dataset/test',
        target_size=(150, 150),
        batch_size=10)

Found 1315 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


In [3]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,strides=(1,1),padding='same',activation='relu',input_shape=(150,150,3)))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=2,padding='same'))
cnn.add(tf.keras.layers.Conv2D(filters=128,kernel_size=3,strides=(1,1),padding='same',activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=2,padding='same'))
cnn.add(tf.keras.layers.Conv2D(filters=256,kernel_size=3,strides=(1,1),padding='same',activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=2,padding='same'))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dropout(0.5))
cnn.add(tf.keras.layers.Dense(128,activation='relu'))
cnn.add(tf.keras.layers.Dense(2,activation='sigmoid'))
cnn.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [4]:
#cnn.fit(x=train_generator,validation_data=test_generator,epochs=20)

In [5]:
#TO DOWNLOAD THE CHECKPOINT FROM GOOGLE COLAB TO LOCAL SYSTEM
#from google.colab import files
#files.download('/content/drive/My Drive/Project Files/CNN Mask Detection/Mask Detection.ipynb')

In [6]:
#To save the trained model
#cnn.save('mask_recog_ver3.h5')

In [7]:
#To load model
from keras.models import load_model
cnn = load_model("mask_recog_ver3.h5")

In [8]:
from cv2 import data

In [9]:
import cv2
import numpy as np
from keras.models import load_model
#model=load_model("./model-010.h5")

results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}

rect_size = 4
cap = cv2.VideoCapture(0) 


haarcascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
#haarcascade = cv2.CascadeClassifier('/home/user_name/.local/lib/python3.6/site-packages/cv2/data/haarcascade_frontalface_default.xml')

while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 

    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=cnn.predict(reshaped)

        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break

cap.release()

cv2.destroyAllWindows()

In [10]:
cv2.__version__

'4.4.0'